In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import time
from sklearn.preprocessing import StandardScaler 
from sklearn.feature_selection import VarianceThreshold
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, f1_score
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE

In [2]:
df_original = pd.read_csv('train_data.csv')

In [3]:
df_rf = df_original.copy()

In [4]:
le = LabelEncoder()
df_rf['profession']= le.fit_transform(df_rf['profession'])
df_rf['gender']= le.fit_transform(df_rf['gender'])
df_rf['highestDegree']= le.fit_transform(df_rf['highestDegree'])
df_rf['maritalStatus']= le.fit_transform(df_rf['maritalStatus'])
df_rf['noOfKids']= le.fit_transform(df_rf['noOfKids'])
df_rf['otherMembership']= le.fit_transform(df_rf['otherMembership'])
df_rf['blogger']= le.fit_transform(df_rf['blogger'])
df_rf['articles']= le.fit_transform(df_rf['articles'])
df_rf['amexCard']= le.fit_transform(df_rf['amexCard'])
df_rf['purposeTravel']= le.fit_transform(df_rf['purposeTravel'])

# y_rf = le.fit_transform(df_rf['loyaltyClass'])
# df_rf = df_rf.drop(columns=['loyaltyClass'], axis=1)

In [17]:
df_rf['age'].fillna(df_rf['age'].median(), inplace=True)
df_rf['gender'].fillna(df_rf['gender'].mode(), inplace=True)
df_rf['noOfKids'].fillna(df_rf['noOfKids'].median(), inplace=True)
df_rf['pastStays1y'].fillna(df_rf['pastStays1y'].median(), inplace=True)
df_rf['gender'] = df_rf['gender'].fillna(df_rf['gender'].mode()[0])
df_rf['profession'] = df_rf['profession'].fillna(df_rf['profession'].mode()[0])

In [6]:
df_rf = df_rf.drop(columns=['ID'])
le = LabelEncoder()
y_all = le.fit_transform(df_rf['loyaltyClass'])
df_rf = df_rf.drop(columns=['loyaltyClass'], axis=1)
x_all = df_rf

In [7]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda_model = LinearDiscriminantAnalysis()
X_lda = lda_model.fit_transform(x_all, y_all)

In [8]:
x, x_test, y, y_test = train_test_split(X_lda, y_all, test_size=0.20, random_state=42)

In [9]:
sm = SMOTE()
x_train_res, y_train_res = sm.fit_resample(x, y.ravel())

In [10]:
clf = XGBClassifier(n_estimator=500, learning_rate=0.1)

In [11]:
start_time= time.time()
# model = clf.fit(x_train_res, y_train_res)
model = clf.fit(x, y)
print("___ %s seconds ---" % (time.time() - start_time))

/Users/chandanshashwat/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:32:02] WARNING: ../src/learner.cc:576: 
Parameters: { "n_estimator" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[11:32:02] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
___ 0.7536540031433105 seconds ---


In [12]:
predictions = clf.predict_proba(x_test)

In [13]:
y_classes = predictions.argmax(axis=-1)

In [14]:
print('Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_classes)))

Model accuracy score: 0.6005


In [15]:
predictions = clf.predict_proba(x_test)
y_classes = predictions.argmax(axis=-1)
cf_matrix = confusion_matrix(y_test, y_classes)
cf_matrix

array([[248,   1, 529],
       [ 18,   0,  57],
       [191,   3, 953]])

In [16]:
#feature selection in case we can get more recall if drop classes 
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

In [17]:
x, x_test, y, y_test = train_test_split(x_all, y_all, test_size=0.20, random_state=42)

In [17]:
sel = SelectFromModel(RandomForestClassifier(n_estimators = 100))
sel.fit(x, y)

SelectFromModel(estimator=RandomForestClassifier())

In [18]:
sel.get_support()

array([ True, False,  True, False,  True, False,  True])

In [19]:
selected_feat= x.columns[(sel.get_support())]
len(selected_feat)

4

In [20]:
print(selected_feat)

Index(['age', 'creditRisk', 'articles', 'staySpend'], dtype='object')


In [9]:
x_all

,profession,age,gender,highestDegree,maritalStatus,noOfKids,creditRisk,otherMembership,pastStays1y,blogger,articles,amexCard,purposeTravel,staySpend
0,2,39.0,1,3,2,0,3.18,3,23.0,0,7,1,2,1.433148e+03
1,1,39.0,2,0,1,1,2.07,3,48.0,0,4,1,1,3.837902e+01
2,0,44.0,0,5,1,5,2.81,0,32.0,0,1,0,1,0.000000e+00
3,2,39.0,1,1,2,0,3.46,3,6.0,0,16,1,2,0.000000e+00
4,0,39.0,0,1,3,5,3.21,5,25.0,1,0,0,0,7.644188e+02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,4,39.0,0,1,1,5,2.55,3,12.0,1,0,0,1,0.000000e+00
9996,1,39.0,0,3,1,1,2.73,3,9.0,0,4,1,2,9.387970e-07
9997,1,58.0,2,4,2,0,3.02,0,48.0,0,5,0,0,3.118682e+02
9998,1,27.0,0,3,2,0,1.50,4,40.0,0,4,0,0,8.269680e-05
